# Dolphin Data

- Read in and plot dolphin data
- There are 3 dolphins, do we want to create different output data for each dolphin?
- Dolphin IDs are 118181 (female - Lizzy), 118182 (male - Perry ), 118183(male - Bent High Nicks)


In [1]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import cartopy.feature as cfeature

#import configuration location and filepath
from myconfig import *

plt.rcParams['figure.figsize'] = (10,10)

def calculate_direction(lat, lon, center_lat, center_lon):
    dlat = lat - center_lat
    dlon = lon - center_lon
    
    if dlat > 0 and dlon == 0:
        return 'N'
    elif dlat > 0 and dlon > 0:
        return 'NE'
    elif dlat == 0 and dlon > 0:
        return 'E'
    elif dlat < 0 and dlon > 0:
        return 'SE'
    elif dlat < 0 and dlon == 0:
        return 'S'
    elif dlat < 0 and dlon < 0:
        return 'SW'
    elif dlat == 0 and dlon < 0:
        return 'W'
    elif dlat > 0 and dlon < 0:
        return 'NW'
    else:
        return None

# Step 6: Function to calculate Euclidean distance between two lat/lon points
def calculate_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2)


filename = './../Dolphin/DS Sarasota 2012 Combined-Locations.csv'

df = pd.read_csv(filename)
ds = xr.Dataset.from_dataframe(df)

#drop low quality observations
mask = ~ds.Quality.isin(['A', 'B', 'G'])
ds = ds.where(mask,drop=True)
ds

#convert the time to np.datetime64
tem_time = pd.to_datetime(ds['Date'].values, format='%H:%M:%S %d-%b-%Y')
tem_time = tem_time.astype('datetime64[ns]')

ds2 = xr.Dataset(
    {
        'DeployID': ('time', ds.DeployID.data),
        'Latitude': ('time', ds.Latitude.data),
        'Longitude': ('time', ds.Longitude.data)
    },
    coords={'time': tem_time}
)

ds = ds2
ds

<xarray.Dataset> Size: 61kB
Dimensions:    (time: 1904)
Coordinates:
  * time       (time) datetime64[ns] 15kB 2012-05-07T14:09:04 ... 2012-08-15T...
Data variables:
    DeployID   (time) float64 15kB 1.182e+05 1.182e+05 ... 1.182e+05 1.182e+05
    Latitude   (time) float64 15kB 27.48 27.48 27.5 27.47 ... 27.51 27.49 27.51
    Longitude  (time) float64 15kB -82.69 -82.7 -82.65 ... -82.55 -82.67 -82.68

In [ ]:
# Step 3: Create a plot with a projection using Cartopy
fig, ax = plt.subplots(figsize=(10, 10),
                       subplot_kw={'projection': ccrs.PlateCarree()})

# Step 4: Plot the data points on the map
ax.scatter(ds['Longitude'], ds['Latitude'], c='blue', s=10, label='Data Points', transform=ccrs.PlateCarree())

# Step 5: Add coastlines to the map
ax.coastlines(resolution='110m')

# Optionally add more map features (borders, land, oceans, etc.)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(cfeature.OCEAN)

# Step 6: Resize the map to the region of the points
ax.set_extent([ds['Longitude'].min(), ds['Longitude'].max(),
               ds['Latitude'].min(), ds['Latitude'].max()],
              crs=ccrs.PlateCarree())

# Adding labels and title
plt.title('Map of Lat/Long Points with Coastlines')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.savefig('./figures/dolphin_map.png')

In [ ]:
# Calculate the center point (mean latitude and longitude)
center_lat = ds['Latitude'].mean().item()
center_lon = ds['Longitude'].mean().item()
center_point = (center_lat, center_lon)

In [ ]:
# Calculate distance of each point from the center point (Euclidean distance)
ds['distance'] = np.sqrt((ds['Latitude'] - center_lat)**2 + (ds['Longitude'] - center_lon)**2)

# Determine direction for each point
directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
farthest_points = {}

# Initialize a dictionary to store farthest points in each direction
for direction in directions:
    farthest_points[direction] = {'lat': None, 'lon': None, 'distance': -1}

# Iterate over the dataset and find the farthest point in each direction
#for lat, lon, distance in zip(ds['Latitude'].values, ds['Longitude'].values, ds['distance'].values):
#    direction = calculate_direction(lat, lon, center_lat, center_lon)
#    if direction and distance > farthest_points[direction]['distance']:
#        farthest_points[direction] = {'lat': lat, 'lon': lon, 'distance': distance}

index = ds['Latitude'].argmax().item()
farthest_points['N'] = {'lat': ds.Latitude[index].item(), 'lon': center_lon, 'distance': ds.distance[index].item()}
index = ds['Latitude'].argmin().item()
farthest_points['S'] = {'lat': ds.Latitude[index].item(), 'lon': center_lon, 'distance': ds.distance[index].item()}
index = ds['Longitude'].argmax().item()
farthest_points['E'] = {'lat': center_lat, 'lon': ds.Longitude[index].item(), 'distance': ds.distance[index].item()}
index = ds['Longitude'].argmin().item()
farthest_points['W'] = {'lat': center_lat, 'lon': ds.Longitude[index].item(), 'distance': ds.distance[index].item()}

#NE
index = ds['Latitude'].argmax().item()
lat2 = center_lat+.5*(ds.Latitude[index].item()-center_lat)
index = ds['Longitude'].argmax().item()
lon2 = center_lon+.5*(ds.Longitude[index].item()-center_lon)
dist2 = np.sqrt((lat2 - center_lat)**2 + (lon2 - center_lon)**2)
farthest_points['NE'] = {'lat': lat2, 'lon': lon2, 'distance': dist2}

#SE
index = ds['Latitude'].argmin().item()
lat2 = center_lat+.5*(ds.Latitude[index].item()-center_lat)
index = ds['Longitude'].argmax().item()
lon2 = center_lon+.5*(ds.Longitude[index].item()-center_lon)
dist2 = np.sqrt((lat2 - center_lat)**2 + (lon2 - center_lon)**2)
farthest_points['SE'] = {'lat': lat2, 'lon': lon2, 'distance': dist2}

#NW
index = ds['Latitude'].argmax().item()
lat2 = center_lat+.5*(ds.Latitude[index].item()-center_lat)
index = ds['Longitude'].argmin().item()
lon2 = center_lon+.5*(ds.Longitude[index].item()-center_lon)
dist2 = np.sqrt((lat2 - center_lat)**2 + (lon2 - center_lon)**2)
farthest_points['NW'] = {'lat': lat2, 'lon': lon2, 'distance': dist2}

#SW
index = ds['Latitude'].argmin().item()
lat2 = center_lat+.5*(ds.Latitude[index].item()-center_lat)
index = ds['Longitude'].argmin().item()
lon2 = center_lon+.5*(ds.Longitude[index].item()-center_lon)
dist2 = np.sqrt((lat2 - center_lat)**2 + (lon2 - center_lon)**2)
farthest_points['SW'] = {'lat': lat2, 'lon': lon2, 'distance': dist2}

# Output the farthest points
for direction, point in farthest_points.items():
    print(f"Farthest point to the {direction}: (Latitude: {point['lat']}, Longitude: {point['lon']}, Distance: {point['distance']:.4f})")

In [ ]:
# Create a map using Cartopy
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot coastlines
ax.coastlines()

lons, lats = ds.Longitude,ds.Latitude
# Plot the data points
plt.scatter(lons, lats, color='blue', s=10, label='Data Points')

# Plot the center point
plt.scatter(center_lon, center_lat, color='red', s=100, marker='*', label='Center Point')

# Plot the farthest points in each direction
for direction, point in farthest_points.items():
    plt.scatter(point['lon'], point['lat'], color='green', s=50, label=f'Farthest {direction}')

# Step 5: Set the map extent to the data region
lat_min, lat_max = lats.min(), lats.max()
lon_min, lon_max = lons.min(), lons.max()
ax.set_extent([lon_min - .02, lon_max + .02, lat_min - .02, lat_max + .02], crs=ccrs.PlateCarree())

# Step 6: Add labels and a legend
plt.title('Map of Data Points with Center and Farthest Points')
plt.legend()

# Display the map
plt.savefig('./figures/dolphin_map_with_8_points.png')

In [ ]:
# Calculate distance for each point and direction
for lat, lon, distance in zip(ds['Latitude'].values, ds['Longitude'].values, ds['distance'].values):
    direction = calculate_direction(lat, lon, center_lat, center_lon)
    if direction and distance > farthest_points[direction]['distance']:
        farthest_points[direction] = {'lat': lat, 'lon': lon, 'distance': distance}


# For each location, find the closest farthest point
closest_farthest = []

for lat, lon in zip(ds['Latitude'].values, ds['Longitude'].values):
    min_distance = float('inf')
    closest_direction = None
    
    for direction, point in farthest_points.items():
        distance = calculate_distance(lat, lon, point['lat'], point['lon'])
        if distance < min_distance:
            min_distance = distance
            closest_direction = direction
    
    closest_farthest.append(closest_direction)

# Add the closest farthest point to the dataset
ds['closest_farthest_point'] = xr.DataArray(closest_farthest, dims='index')

# Output the dataset with the closest farthest points
print(ds[['Latitude', 'Longitude', 'closest_farthest_point']])

In [ ]:
# Plot the closest farthest point as a time series
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(ds['time'], ds['closest_farthest_point'], marker='o', linestyle='-', label='Closest Farthest Point')

# Add labels and a title
plt.title('Time Series of Closest Farthest Point')
plt.xlabel('Time')
plt.ylabel('Closest Farthest Point')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Show the legend
plt.legend()
plt.savefig('./figures/dolphin_map_timeseries.png')

In [ ]:
# Color mapping for each farthest point direction
color_map = {
    'N': 'cyan',
    'NE': 'magenta',
    'E': 'orange',
    'SE': 'yellow',
    'S': 'green',
    'SW': 'blue',
    'W': 'red',
    'NW': 'purple'
}
# Step 4: Create a map using Cartopy
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot coastlines
ax.coastlines()

lons, lats = ds.Longitude,ds.Latitude
# Plot the data points
plt.scatter(lons, lats, color='blue', s=10, label='Data Points')

# Plot the center point
plt.scatter(center_lon, center_lat, color='red', s=100, marker='*', label='Center Point')

# Plot the farthest points in each direction
for direction, point in farthest_points.items():
    plt.scatter(point['lon'], point['lat'], color='green', s=50, label=f'Farthest {direction}')

closest_farthest_points = ds['closest_farthest_point'].values  # Assuming this is available
colors = [color_map[direction] for direction in closest_farthest_points]

ax.scatter(ds['Longitude'], ds['Latitude'], color=colors, marker='o', label='Data Points', transform=ccrs.PlateCarree())

# Set the map extent to the data region
lat_min, lat_max = lats.min(), lats.max()
lon_min, lon_max = lons.min(), lons.max()
ax.set_extent([lon_min - .02, lon_max + .02, lat_min - .02, lat_max + .02], crs=ccrs.PlateCarree())

# Add labels and a legend
plt.title('Map of Data Points with Center and Farthest Points')
plt.legend()

# Display the map
plt.savefig('./figures/dolphin_map_mapped_to_point.png')


In [ ]:
#mask
mask = ds.DeployID==118181
ds2 = ds.where(mask,drop=True)
nc_fname = output_path2+'dolphin_Lizzy.nc'
csv_fname = output_path2+'dolphin_Lizzy.csv'
ds2.to_netcdf(nc_fname) 
df = ds2.to_dataframe()
df.to_csv(csv_fname)

mask = ds.DeployID==118182
ds2 = ds.where(mask,drop=True)
nc_fname = output_path2+'dolphin_Perry.nc'
csv_fname = output_path2+'dolphin_Perry.csv'
ds2.to_netcdf(nc_fname) 
df = ds2.to_dataframe()
df.to_csv(csv_fname)

mask = ds.DeployID==118183
ds2 = ds.where(mask,drop=True)
nc_fname = output_path2+'dolphin_BentHighNick.nc'
csv_fname = output_path2+'dolphin_BentHighNick.csv'
ds2.to_netcdf(nc_fname) 
df = ds2.to_dataframe()
df.to_csv(csv_fname)

In [ ]:

# Color mapping for each farthest point direction
color_map = {
    'N': 'cyan',
    'NE': 'magenta',
    'E': 'orange',
    'SE': 'yellow',
    'S': 'green',
    'SW': 'blue',
    'W': 'red',
    'NW': 'purple'
}
# Step 4: Create a map using Cartopy
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot coastlines
ax.coastlines()
# Add features to the map
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)


mask = ds.DeployID==118182
ds2 = ds.where(mask,drop=True)
lons, lats = ds2.Longitude,ds2.Latitude
ax.plot(lons,lats, marker='o', color='blue',  linewidth=.25, markersize=2, transform=ccrs.PlateCarree())

mask = ds.DeployID==118183
ds2 = ds.where(mask,drop=True)
lons, lats = ds2.Longitude,ds2.Latitude
ax.plot(lons,lats, marker='o', color='green',  linewidth=.25, markersize=2, transform=ccrs.PlateCarree())

mask = ds.DeployID==118181
ds2 = ds.where(mask,drop=True)
lons, lats = ds2.Longitude,ds2.Latitude
ax.plot(lons,lats, marker='o', color='pink', linewidth=.25, markersize=2, transform=ccrs.PlateCarree())

# Set the map extent to the data region
lons, lats = ds.Longitude,ds.Latitude
lat_min, lat_max = lats.min(), lats.max()
lon_min, lon_max = lons.min(), lons.max()
ax.set_extent([lon_min + .02, lon_max + .02, lat_min - .02, lat_max + .02], crs=ccrs.PlateCarree())

plt.text(lon_min+.05,lat_min+.1,'Lizzy', color='pink',fontsize=14,weight='bold')
plt.text(lon_min+.05,lat_min+.08,'Perry', color='blue',fontsize=14,weight='bold')
plt.text(lon_min+.05,lat_min+.06,'Bent-High-Nick', color='green',fontsize=14,weight='bold')
plt.savefig('./figures/dolphin_tracks.png')

In [2]:
# creating a single table with all the dolphins
# that has a column for angular direction and distance from the center lat/lon
# interpolate onto evenly space time intervals
# so 0 where no ping and then each dolphin is a separate column

import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import cartopy.feature as cfeature

#import configuration location and filepath
from myconfig import *

def calculate_direction_angular(lat, lon, center_lat, center_lon):
    dlat = lat - center_lat
    dlon = lon - center_lon
    angle = np.arctan([dlat, dlon])*np.pi/180
    return angle

# Step 6: Function to calculate Euclidean distance between two lat/lon points
def calculate_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2)

filename = './../Dolphin/DS Sarasota 2012 Combined-Locations.csv'
df = pd.read_csv(filename)
ds = xr.Dataset.from_dataframe(df)

#drop low quality observations
mask = ~ds.Quality.isin(['A', 'B', 'G'])
ds = ds.where(mask,drop=True)
ds

#convert the time to np.datetime64
tem_time = pd.to_datetime(ds['Date'].values, format='%H:%M:%S %d-%b-%Y')
tem_time = tem_time.astype('datetime64[ns]')

ds2 = xr.Dataset(
    {
        'DeployID': ('time', ds.DeployID.data),
        'Latitude': ('time', ds.Latitude.data),
        'Longitude': ('time', ds.Longitude.data)
    },
    coords={'time': tem_time}
)

In [3]:
filename = './../Dolphin/DS Sarasota 2012 Combined-Locations.csv'
df = pd.read_csv(filename)
ds = xr.Dataset.from_dataframe(df)
#drop low quality observations
mask = ~ds.Quality.isin(['A', 'B', 'G'])
ds = ds.where(mask,drop=True)
# Calculate the center point (mean latitude and longitude)
center_lat = ds['Latitude'].mean().item()
center_lon = ds['Longitude'].mean().item()
center_point = (center_lat, center_lon)
print(center_point)
def calculate_direction_angular(lat, lon, center_lat, center_lon):
    lat = np.radians(lat)
    lon = np.radians(lon)
    center_lat = np.radians(center_lat)
    center_lon = np.radians(center_lon)
    #dlat = lat - center_lat
    #dlon = lon - center_lon
    #angle = np.atan2([dlon*np.pi/180,dlat*np.pi/180.])*180/np.pi
    #print(dlat,lat,center_lat)
    angle = np.atan2(lon-center_lon,lat-center_lat)
    angle = np.degrees(angle)
    #angle = angle + 90
    angle = (angle + 360) % 360
    return angle

(27.483218498461138, -82.6416578211397)


In [4]:
name=['Lizzy','Perry','BentHighNick']
for ifile in range(3):
    nc_fname = output_path2+'dolphin_'+name[ifile]+'.nc'
    print(nc_fname)
    ds2 = xr.open_dataset(nc_fname)
    ds2 = ds2.drop_vars('closest_farthest_point')
    ds2['angle'] = calculate_direction_angular(ds2.Latitude,ds2.Longitude, center_lat, center_lon)
    start_time = pd.to_datetime(ds2.time[0].data).strftime('2012-05-07 13:30:00')
    end_time = pd.to_datetime(ds2.time[-1].data).strftime('2012-08-15 08:30:00')
    print(start_time,end_time)
    # Generate the time intervals with 15-minute frequency
    time_intervals = pd.date_range(start=start_time, end=end_time, freq='15min')
    ds_rounded = ds2.assign_coords(time=ds2['time'].dt.round('15min')) #round to 15 min
    ds_unique = ds_rounded.isel(time=~ds_rounded.get_index("time").duplicated()) #remove duplicates
    ds_regular = ds_unique.reindex({'time': time_intervals}, fill_value=np.nan)
    nc_fname = output_path2+'dolphin_'+name[ifile]+'_angle.nc'
    csv_fname = output_path2+'dolphin_'+name[ifile]+'_angle.csv'
    ds_regular.to_netcdf(nc_fname) 
    df = ds_regular.to_dataframe()
    df.to_csv(csv_fname)

./../dolphin/dolphin_Lizzy.nc
2012-05-07 13:30:00 2012-08-15 08:30:00
./../dolphin/dolphin_Perry.nc
2012-05-07 13:30:00 2012-08-15 08:30:00
./../dolphin/dolphin_BentHighNick.nc
2012-05-07 13:30:00 2012-08-15 08:30:00


In [6]:
new_data = xr.Dataset()
name=['Lizzy','Perry','BentHighNick']
for ifile in range(3):
    nc_fname = output_path2+'dolphin_'+name[ifile]+'.nc'
    print(nc_fname)
    ds2 = xr.open_dataset(nc_fname)
    ds2 = ds2.drop_vars('closest_farthest_point')
    ds2['angle'] = calculate_direction_angular(ds2.Latitude,ds2.Longitude, center_lat, center_lon)
    start_time = pd.to_datetime(ds2.time[0].data).strftime('2012-05-07 13:30:00')
    end_time = pd.to_datetime(ds2.time[-1].data).strftime('2012-08-15 08:30:00')
    print(start_time,end_time)
    # Generate the time intervals with 15-minute frequency
    time_intervals = pd.date_range(start=start_time, end=end_time, freq='15min')
    ds_rounded = ds2.assign_coords(time=ds2['time'].dt.round('15min')) #round to 15 min
    ds_unique = ds_rounded.isel(time=~ds_rounded.get_index("time").duplicated()) #remove duplicates
    ds_regular = ds_unique.reindex({'time': time_intervals}, fill_value=np.nan)
    newstr = name[ifile]+'_angle'
    #ds_regular = ds_regular.rename({'angle':newstr})
    # Step 3: Copy the 'temperature' variable to the new dataset
    new_data[newstr] = ds_regular['angle']
nc_fname = output_path2+'dolphin_angle.nc'
csv_fname = output_path2+'dolphin_angle.csv'
new_data.to_netcdf(nc_fname) 
df = new_data.to_dataframe()
df.to_csv(csv_fname)

./../dolphin/dolphin_Lizzy.nc
2012-05-07 13:30:00 2012-08-15 08:30:00
./../dolphin/dolphin_Perry.nc
2012-05-07 13:30:00 2012-08-15 08:30:00
./../dolphin/dolphin_BentHighNick.nc
2012-05-07 13:30:00 2012-08-15 08:30:00


In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})
angles_rad = new_data.Lizzy_angle.data
ax.scatter(angles_rad, np.ones_like(angles_rad), s=100, color='blue', alpha=0.7, edgecolors='k')


In [ ]:
ds_rounded.time.plot()

In [ ]:
#ds2.DeployID.plot()
ds_rounded.DeployID.plot()

In [ ]:
dd= ds2.time[0:-2].data-ds2.time[1:-1].data
plt.plot(dd)

In [ ]:
ds2.time[0:100]
dd[676]

In [ ]:
ds2.time[676:678].data